In [2]:
import numpy as np
import control as ctrl

from OrderUncertainties import *
from Benchmarks import sys_variables


In [3]:
def get_sensitivity(A, B):
    mat = OrdUnc(A)
    if mat.determineCase() == 1:
        unc = mat.distinctPos(B)
    else:
        unc = mat.multSig(B)
    return unc

def all_sensitivity(bench, K, openloop=False):
    # A_bar_i = A + BKE_i where E_i = I(n) with the (i, i) entry swapped to lambda
    # A_bar_i = AE_i where E_i = I(n) with the (i, i) entry swapped to lambda
    if openloop:
        X = bench.A
        Y = bench.A
    else:
        X = bench.A + bench.B @ K
        Y = bench.B @ K

    sensitivity = []
    for i in range(bench.nx):
        Yi = np.zeros((bench.nx, bench.nx))
        Yi[:, i] = Y[:, i]
        sensitivity.append(get_sensitivity(X, Yi))

    return sensitivity


In [4]:
res = {}
for (name, bench) in sys_variables.items():
    K = ctrl.lqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
    res[name] = list(map(lambda x: float(f"{x:.2g}"), all_sensitivity(bench, K)))
res

{'RC': [-0.059, 0.24],
 'F1': [8.5, -2.4],
 'DC': [0.00014, -0.00092],
 'CS': [54.0, 12.0, 16.0, 0.4],
 'EW': [9700.0, -3.4e-05],
 'C1': [-0.00099],
 'CC': [0.066, 0.14, -0.19],
 'D5': [0.0, 0.0, 0.0, 0.0, 0.0]}

In [5]:
sys_variables['D5'].A.round(4)

array([[ -4.8082,  -4.0336,   1.561 ,  -1.9059,  -0.7299],
       [ -1.4137,  -7.2405,  -1.4615,  -3.4139,   4.4378],
       [  7.1423,  12.4581,   0.348 ,   8.6344,  -6.9834],
       [ -0.0349,   2.3922,   0.8171,   0.0909,  -3.5798],
       [ -6.6104, -14.507 ,   1.1341,  -7.1915,   1.6098]])

In [6]:
np.linalg.det(sys_variables['D5'].A)

-340.00000000000074

In [7]:
list(map(lambda x: float(f"{x:.2g}"), all_sensitivity(sys_variables['D5'], [], True)))

[3.4, 9.8, 0.64, 1.7, 3.4]